<a href="https://colab.research.google.com/github/nittayac/DFEDATA6-EX1/blob/main/Unstruct_to_Struct/readEmail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os

In [ ]:
# account credentials
username = "nittayanatt@hotmail.com"
password = ""

In [ ]:
def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

In [ ]:
# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL("imap-mail.outlook.com") #outlook and hotmail
#imap = imaplib.IMAP4_SSL("imap.gmail.com") # gmail
# authenticate
imap.login(username, password)

('OK', [b'LOGIN completed.'])

In [ ]:
status, messages = imap.select("INBOX")  #selects a mailbox (Inbox, spam, etc.),
# number of top emails to fetch
N = 3
# total number of emails
messages = int(messages[0])
print(messages)

7


In [ ]:
for i in range(messages, messages-N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")    #imap.fetch() method, which fetches the email message by ID using the standard format specified in RFC 822.
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]  #decode_header() function from the email.header module to decode the subject of the email address to human-readable Unicode.
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode(encoding)
            # decode email sender
            From, encoding = decode_header(msg.get("From"))[0]
            if isinstance(From, bytes):
                From = From.decode(encoding)
            print("Subject:", subject)
            print("From:", From)
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        print(body)
                    elif "attachment" in content_disposition:
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            folder_name = clean(subject)
                            if not os.path.isdir(folder_name):
                                # make a folder for this email (named after the subject)
                                os.mkdir(folder_name)
                            filepath = os.path.join(folder_name, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                if content_type == "text/plain":
                    # print only text email parts
                    print(body)
            if content_type == "text/html":
                # if it's HTML, create a new HTML file and open it in browser
                folder_name = clean(subject)
                if not os.path.isdir(folder_name):
                    # make a folder for this email (named after the subject)
                    os.mkdir(folder_name)
                filename = "index.html"
                filepath = os.path.join(folder_name, filename)
                # write the file
                open(filepath, "w").write(body)
                # open in the default browser
                webbrowser.open(filepath)
            print("="*100)
# close the connection and logout
imap.close()
imap.logout()

Subject: Microsoft account security info verification
From: Microsoft account team
	<account-security-noreply@accountprotection.microsoft.com>
You recently verified the security info on the Microsoft account ni**t@hotmail.com. This was a periodic security check which only happens if you haven't used a security code recently. You won't need to provide a code every time you sign in.

It's important to keep the security info associated with your account correct and up to date. We'll never use this info to spam you or for any marketing purposes – it's only to verify your identity if there's ever a problem with your account.

To learn more or contact support, click here http://go.microsoft.com/fwlink/?LinkID=281822.

To opt out or change where you receive security notifications, click here https://account.live.com/SecurityNotifications/Update.

Thanks,
The Microsoft account team 
Subject: Microsoft account security info was added
From: Microsoft account team
	<account-security-noreply@accou

('BYE', [b'Microsoft Exchange Server IMAP4 server signing off.'])